In [ ]:
# This cell is nessesary only if you excutes the model in google colab
# Otherways comment it
!pip uninstall -y kaggle
!pip install kaggle==1.5.6
%env KAGGLE_USERNAME = bassamsabber
%env KAGGLE_KEY = 717678f8ebfb53d84e9c47444c275871
!kaggle competitions download -c jigsaw-toxic-comment-classification-challenge
!unzip jigsaw-toxic-comment-classification-challenge
!unzip test.csv
!unzip train.csv

In [ ]:
import numpy as np
import pandas as pd

from keras.models import Model, Sequential
from keras.layers import Dense,Embedding, Input, Activation
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import initializers, optimizers, layers
from sklearn.metrics import roc_auc_score


In [ ]:
# importing data if using colab
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')
#train.head()
#test.head()
#train['comment_text'][0]

In [ ]:
classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
targets = train[classes].values
#print(targets)

train_sentences = train['comment_text']
test_sentences = test['comment_text']

In [ ]:
max_features = 22000
tokenizer = Tokenizer(num_words = max_features)
tokenizer.fit_on_texts(list(train_sentences))

tokenized_train = tokenizer.texts_to_sequences(train_sentences)
tokenized_test = tokenizer.texts_to_sequences(test_sentences)

In [ ]:
maxlen = 200
X_train = pad_sequences(tokenized_train, maxlen = maxlen)
X_test = pad_sequences(tokenized_test, maxlen = maxlen)

In [ ]:
embed_size = 128
maxlen = 200
max_features = 22000

input = Input(shape = (maxlen, ))
output_net = Embedding(max_features, embed_size)(input)
output_net = LSTM(60, return_sequences=True, name='lstm_layer')(output_net)
output_net = GlobalMaxPool1D()(output_net)
output_net = Dropout(0.1)(output_net)
output_net = Dense(50, activation="relu")(output_net)
output_net = Dropout(0.1)(output_net)
output_net = Dense(6, activation="sigmoid")(output_net)

In [ ]:
model = Model(inputs=input, outputs=output_net)
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
model.summary()

In [ ]:
batch_size = 64
epochs = 2
model.fit(X_train, targets, batch_size=batch_size, epochs=epochs, validation_split=0.1)
prediction = model.predict(X_train)
print("Training Accuracy:", np.round(roc_auc_score(targets, prediction)*100, 1), "%")

In [ ]:
prediction = model.predict(X_test)
#prediction

In [ ]:
sample = ["I like meat and milk","Good morning mother fucker"]
tokenizers = Tokenizer(num_words = max_features)
tokenizers.fit_on_texts(list(train_sentences))
tokenized = tokenizers.texts_to_sequences(sample)
case = pad_sequences(tokenized, maxlen=200) 
print(case)
mdl = model.predict(case)
#print(mdl)
#print(X_train[0])
#print(roc_auc_score(targets, prediction))

In [ ]:
#!unzip sample_submission.csv
#submit = pd.read_csv('/content/sample_submission.csv')
#submit[classes] = prediction
#submit.to_csv('/content/sample_submission.csv')


